<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'asl-ml-immersion-temp'
PROJECT = 'asl-ml-immersion'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [1]:
import google.datalab.bigquery as bq
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  mother_race,
  plurality,
  gestation_weeks,
  mother_married,
  ever_born,
  cigarette_use,
  alcohol_use,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [2]:
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,1922480672677032316,367231
1,411066950820961322,318834
2,1569531340167098963,348504
3,6782146986770280327,345092
4,4979697502521811334,357965


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [11]:
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13369 examples in the train dataset and 3384 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Notice that the race field is 1.0, 4.0, etc.  Let's replace by text-strings. Our final deployed service will ask for text names, not magic numbers.

In [5]:
traindf.head()

,weight_pounds,is_male,mother_age,mother_race,plurality,gestation_weeks,mother_married,cigarette_use,alcohol_use,hashmonth
0,8.518662,True,21,White,1,NaN,True,None,None,774501970389208065
1,8.595824,False,29,White,1,39.0,True,None,None,774501970389208065
2,6.499227,False,34,White,1,40.0,True,None,None,774501970389208065
3,8.849355,False,24,White,1,40.0,True,None,None,774501970389208065
4,7.936641,True,26,White,1,40.0,False,None,None,774501970389208065


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [8]:
traindf.describe()

,weight_pounds,mother_age,mother_race,plurality,gestation_weeks,ever_born,hashmonth
count,13179.000000,13188.000000,9305.000000,13188.000000,13110.000000,13140.000000,1.318800e+04
mean,7.231032,27.429785,2.872434,1.034198,38.602517,2.075875,4.402066e+18
std,1.338385,6.139551,9.738043,0.193074,2.579807,1.259573,2.834400e+18
min,0.500449,13.000000,1.000000,1.000000,17.000000,1.000000,1.244589e+17
25%,6.563162,23.000000,1.000000,1.000000,38.000000,1.000000,1.525201e+18
50%,7.312733,27.000000,1.000000,1.000000,39.000000,2.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,1.000000,40.000000,3.000000,7.170970e+18
max,12.908065,48.000000,78.000000,4.000000,47.000000,13.000000,9.183606e+18


In [12]:
def preprocess(df):
  # modify opaque numeric race code into human-readable data
  races = dict(zip([-1,1,2,3,4,5,6,7,18,28,39,48],
                   ['Unknown', 'White', 'Black', 'American Indian', 'Chinese', 
                    'Japanese', 'Hawaiian', 'Filipino',
                    'Asian Indian', 'Korean', 'Samaon', 'Vietnamese']))
  df['mother_race'].fillna(-1, inplace=True)
  df['mother_race'].replace(races, inplace=True)
  
  # remove unwanted columns
  del df['ever_born']
  
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  return df

traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,mother_race,plurality,gestation_weeks,mother_married,cigarette_use,alcohol_use,hashmonth
0,7.813183,True,21,White,1,39.0,False,None,None,774501970389208065
1,7.500126,False,17,White,1,39.0,True,None,None,774501970389208065
2,9.748841,True,19,White,1,39.0,False,None,None,774501970389208065
3,7.936641,False,18,White,1,40.0,False,None,None,774501970389208065
4,7.936641,True,35,White,1,39.0,True,None,None,774501970389208065


In [17]:
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13256.000000,13256.000000,13256.000000,13256.000000,1.325600e+04
mean,7.257200,27.441989,1.033871,38.620172,4.397037e+18
std,1.299769,6.130785,0.190651,2.500767,2.776080e+18
min,0.500449,12.000000,1.000000,18.000000,1.244589e+17
25%,6.602845,23.000000,1.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,7.108882e+18
max,13.558429,49.000000,4.000000,47.000000,9.183606e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [13]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [16]:
%bash
wc -l *.csv
head *.csv

   3352 eval.csv
  13256 train.csv
  16608 total
==> eval.csv <==
7.1870697412,False,19,White,1,38.0,True,,,4740473290291881219
7.5839018128,True,23,Unknown,1,39.0,True,,,4740473290291881219
8.11962510946,True,20,White,1,40.0,True,,,4740473290291881219
4.96481014024,False,43,White,2,36.0,True,,,4740473290291881219
7.87491199864,False,33,Unknown,1,42.0,True,,,4740473290291881219
6.7571683303,True,28,White,1,39.0,True,,,4740473290291881219
8.81187661214,False,39,Unknown,1,39.0,False,,,4740473290291881219
7.3744626639,False,23,Unknown,1,40.0,True,,,4740473290291881219
6.8012607827,False,20,White,1,44.0,True,,,4740473290291881219
6.53009220044,False,21,Unknown,1,40.0,True,,,4740473290291881219

==> train.csv <==
7.81318256528,True,21,White,1,39.0,False,,,774501970389208065
7.50012615324,False,17,White,1,39.0,True,,,774501970389208065
9.74884122564,True,19,White,1,39.0,False,,,774501970389208065
7.936641432,False,18,White,1,40.0,False,,,774501970389208065
7.936641432,True,35,White,1,39.0,Tr

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License